<a href="https://colab.research.google.com/github/ss-ghule/kaggle_predicting_molecular_properties/blob/master/c4_w4_model_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training & Evaluation

## ##Training & Evaluation is currently being carried out in google Colaboratory due to less free resources available in IBM watson studio

# Download Data

In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
from ibm_botocore.client import Config
import ibm_boto3
def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [0]:
download_file_cos(credentials_2,'train_feat_eng.csv','train_feat_eng.csv')
download_file_cos(credentials_2,'test_feat_eng.csv','test_feat_eng.csv')
download_file_cos(credentials_2,'sample_submission.csv','sample_submission.csv')
!ls

File Downloaded
File Downloaded
File Downloaded
sample_submission.csv  test_feat_eng.csv  train_feat_eng.csv


# Creating training, validation and testing data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pickle
%matplotlib inline

In [2]:
tf.__version__

'1.14.0'

In [6]:
train=pd.read_csv('train_feat_eng_v2.csv')
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,distance,type_enc,scalar_coupling_constant
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,0.133921,-0.535689,1.091953,0,84.8076
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,0.133921,0.133922,1.783120,3,-11.2570
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,0.133921,0.133923,1.783147,3,-11.2548
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,0.133921,0.133923,1.783157,3,-11.2543
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,0.133922,-0.535689,1.091952,0,84.8074


In [7]:
train.shape

(4658147, 13)

In [36]:
test=pd.read_csv('test_feat_eng_v2.csv')
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,distance,type_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,NaN,-0.064872,2.261178,2
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,NaN,-0.064872,1.062099,0
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,NaN,0.118187,3.323277,6
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,NaN,-0.064872,1.062099,0
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,NaN,-0.064872,2.261178,2


In [11]:
test.shape

(2505542, 12)

In [12]:
train.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'mulliken_charge_0', 'mulliken_charge_1', 'distance', 'type_enc',
       'scalar_coupling_constant'],
      dtype='object')

In [13]:
x=train.drop('scalar_coupling_constant',axis=1)
x.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,distance,type_enc
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,0.133921,-0.535689,1.091953,0
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,0.133921,0.133922,1.783120,3
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,0.133921,0.133923,1.783147,3
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,0.133921,0.133923,1.783157,3
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,0.133922,-0.535689,1.091952,0


In [14]:
y=train['scalar_coupling_constant']
y.head()

0    84.8076
1   -11.2570
2   -11.2548
3   -11.2543
4    84.8074
Name: scalar_coupling_constant, dtype: float64

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.3)

In [17]:
print('x_train: {}   y_train: {}'.format(x_train.shape,y_train.shape))
print('x_val: {}   y_val: {}'.format(x_val.shape,y_val.shape))

x_train: (3260702, 12)   y_train: (3260702,)
x_val: (1397445, 12)   y_val: (1397445,)


In [18]:
x_test=test
x_test.shape

(2505542, 12)

# Training & Evaluation

In [22]:
input_shape=(12,)
model_nn=keras.Sequential()
model_nn.add(keras.layers.Dense(256,activation='relu',input_shape=input_shape))
model_nn.add(keras.layers.Dense(128,activation='relu'))
model_nn.add(keras.layers.Dense(64,activation='relu'))
model_nn.add(keras.layers.Dense(8,activation='relu'))
model_nn.add(keras.layers.Dense(1))

model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               3328      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 520       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 45,009
Trainable params: 45,009
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_nn.compile(optimizer='adam',loss='mean_absolute_error')

In [24]:
model_nn.fit(x_train,y_train,epochs=5,batch_size=512,validation_data=(x_val,y_val))

Train on 3260702 samples, validate on 1397445 samples
Epoch 1/5
3260702/3260702 [==============================] - 57s 18us/sample - loss: 4.0241 - val_loss: 3.4875
Epoch 2/5
3260702/3260702 [==============================] - 57s 17us/sample - loss: 3.2489 - val_loss: 3.1321
Epoch 3/5
3260702/3260702 [==============================] - 58s 18us/sample - loss: 2.9526 - val_loss: 3.0191
Epoch 4/5
3260702/3260702 [==============================] - 56s 17us/sample - loss: 2.7707 - val_loss: 2.8840
Epoch 5/5
3260702/3260702 [==============================] - 56s 17us/sample - loss: 2.6427 - val_loss: 2.6091


# Train on whole data

In [25]:
model_nn.fit(x,y,epochs=5,batch_size=512,validation_data=(x_val,y_val))

Train on 4658147 samples, validate on 1397445 samples
Epoch 1/5
4658147/4658147 [==============================] - 76s 16us/sample - loss: 2.5600 - val_loss: 2.4878
Epoch 2/5
4658147/4658147 [==============================] - 77s 16us/sample - loss: 2.4934 - val_loss: 2.4628
Epoch 3/5
4658147/4658147 [==============================] - 79s 17us/sample - loss: 2.4412 - val_loss: 2.3996
Epoch 4/5
4658147/4658147 [==============================] - 81s 17us/sample - loss: 2.4006 - val_loss: 2.3612
Epoch 5/5
4658147/4658147 [==============================] - 78s 17us/sample - loss: 2.3657 - val_loss: 2.3981


# Serialize the model

In [0]:
tf.keras.models.save_model(model_nn,'kaggle_model_nn_03.h5',overwrite=True,include_optimizer=True)

In [0]:
!ls

# Create Submission file

In [37]:
import gc
gc.collect()

634

In [38]:
test=pd.read_csv('test_feat_eng_v2.csv')
test_predictions=model_nn.predict(test,verbose=1,batch_size=10000)
test_predictions[:10]

2505542/2505542 [==============================] - 7s 3us/sample


array([[ -3.2472324],
       [ 91.86721  ],
       [ 10.933329 ],
       [107.796974 ],
       [  1.5662003],
       [107.1634   ],
       [  2.08945  ],
       [-11.683802 ],
       [-10.687826 ],
       [109.37023  ]], dtype=float32)

In [39]:
test_predictions.shape

(2505542, 1)

In [40]:
sample_submission=pd.read_csv('sample_submission.csv.zip')
sample_submission.head()

,id,scalar_coupling_constant
0,4658147,0
1,4658148,0
2,4658149,0
3,4658150,0
4,4658151,0


In [41]:
sample_submission['scalar_coupling_constant']=test_predictions
sample_submission.head()

,id,scalar_coupling_constant
0,4658147,-3.247232
1,4658148,91.867210
2,4658149,10.933329
3,4658150,107.796974
4,4658151,1.566200


In [0]:
sample_submission.to_csv('sample_submission_distance_baseline_test.csv',index=False)

In [43]:
!ls

champs-scalar-coupling_new.zip
dipole_moments.csv.zip
kaggle.json
magnetic_shielding_tensors.csv.zip
mulliken_charges.csv.zip
potential_energy.csv.zip
sample_data
sample_submission.csv.zip
sample_submission_distance_baseline_test.csv
scalar_coupling_contributions.csv.zip
structures.csv.zip
structures.zip
test.csv.zip
test_etl.csv
test_feat_eng_v2.csv
train.csv.zip
train_etl.csv
train_feat_eng_v2.csv


In [44]:
!head sample_submission_distance_baseline_test.csv

id,scalar_coupling_constant
4658147,-3.2472324
4658148,91.86721
4658149,10.933329
4658150,107.796974
4658151,1.5662003
4658152,107.1634
4658153,2.08945
4658154,-11.683802
4658155,-10.687826


# Upload to IBM object store

In [0]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [0]:
upload_file_cos(credentials_2,'sample_submission_test_3.csv','sample_submission_test_3.csv')

In [0]:
upload_file_cos(credentials_2,'kaggle_model_nn_03.h5','kaggle_model_nn_03.h5')